In [3]:
import os
import logging
from supabase import create_client
from  dotenv import load_dotenv
from postgrest import APIError
import time
import pandas as pd
load_dotenv()

True

# Test result & latency measurement

In [4]:
from typing import Callable, Any, List, Dict

# Register with supabase
# IMPORTANT to use supabase built-in methods
supabase = create_client(
    os.environ['PROJECT_URL'],
    os.environ['ANON_PUBLIC']
)

def run_experiments_table(experiments: List[Dict[str, Any]]):
    """
    Run a list of experiments and print results in table format.

    Each experiment dict should contain:
        - id: str
        - title: str
        - function: callable
        - args: tuple (optional)
        - kwargs: dict (optional)
    """

    results = []

    for exp in experiments:
        exp_id = exp.get("id", "")
        title = exp.get("title", "")
        func: Callable = exp["function"]
        args = exp.get("args", ())
        kwargs = exp.get("kwargs", {})

        start = time.time()
        status = "SUCCESS"
        return_value = None
        error_msg = ""

        try:
            return_value = func(*args, **kwargs)
        
        except APIError as e:
            status = "FAILED"
            error_msg = e.message
            
        except Exception as e:
            status = "FAILED"
            error_msg = e
            

        latency_ms = (time.time() - start) * 1000

        results.append({
            "ID": exp_id,
            "Experiment": title,
            "Function": func.__name__,
            "Status": status,
            "Latency (ms)": f"{latency_ms:.2f}",
            "Return": str(return_value),
            "Error": error_msg
        })

    # Print table
    headers = results[0].keys()
    col_widths = {
        h: max(len(h), max(len(str(row[h])) for row in results))
        for h in headers
    }

    def print_row(row):
        print(" | ".join(str(row[h]).ljust(col_widths[h]) for h in headers))

    print("\n" + "=" * (sum(col_widths.values()) + 3 * len(headers)))
    print_row({h: h for h in headers})
    print("-" * (sum(col_widths.values()) + 3 * len(headers)))

    for row in results:
        print_row(row)

    print("=" * (sum(col_widths.values()) + 3 * len(headers)))

def measure_latency(func, runs=10, *args, **kwargs):
    latencies = []
    result = None
    for _ in range(runs):
        start = time.time()
        result = func(*args, **kwargs)
        latencies.append((time.time() - start) * 1000)
    return {
        "avg_ms": sum(latencies) / len(latencies),
        "min_ms": min(latencies),
        "max_ms": max(latencies),
        "runs": runs,
        "result": result
    }


# Test cases

In [5]:
def unauthorized_GET(table_name: str):
    response = supabase.table(table_name).select("*").execute()
    return {"row": f"{len(response.data)}"}
    
def authorized_GET(table_name: str, credentials: dict[str, str]):
    # Register to supabase's client object with authorized token
    _session = supabase.auth.sign_in_with_password(credentials)
    supabase.postgrest.auth(_session.session.access_token)
    
    response = supabase.table(table_name).select("*").execute()
    return {"row": f"{len(response.data)}"}

def invalid_auth_GET(table_name: str):
    fake_credentials = {
        "email": "fake@email.com",
        "password": "wrongpassword"
    }
    supabase.auth.sign_in_with_password(fake_credentials)
    response = supabase.table(table_name).select("*").execute()
    return {"row": f"{len(response.data)}"}

def authorized_POST(table_name: str, credentials: dict[str, str], value: dict[str]):
    _session = supabase.auth.sign_in_with_password(credentials)
    supabase.postgrest.auth(_session.session.access_token)
    
    response = supabase.table(table_name).INSERT(value).execute()
    return {"row": f"{len(response.data)}"}
    


# Auth

In [6]:
email = os.environ["email"]
password = os.environ["password"]
credentials = {"email": email, "password": password}
fake_credentials = {"email": "abc@gmail.com", "password": "1234567"}

# Execution

In [8]:
test_case = [
    {
        "id": "5.1.1",
        "title": "SELECT on price table",
        "function": unauthorized_GET,
        "args": ("price",)
    },
    {
        "id": "5.1.2"   ,
        "title": "SELECT on stats table",
        "function": unauthorized_GET,
        "args": ("stats",)
    },
    {
        "id": "5.1.3",
        "title": "SELECT on price table",
        "function": authorized_GET,
        "args": ("price", credentials, )
    },
    {
        "id": "5.1.4"   ,
        "title": "SELECT on stats table",
        "function": authorized_GET,
        "args": ("stats", credentials,)
    },
    {
        "id": "5.1.5"   ,
        "title": "INSERT on price table",
        "function": authorized_POST,
        "args": ("price", credentials, {'ticker': 'ABC'},)
    },
    {
        "id": "5.1.6"   ,
        "title": "INSERT on stats table",
        "function": authorized_POST,
        "args": ("stats", credentials, {'ticker': 'ABC'},)
    },
    {
        "id": "5.1.7",
        "title": "Auth with fake credentials",
        "function": authorized_GET,
        "args": ("price", fake_credentials, )
    },
]

run_experiments_table(test_case)


ID    | Experiment                 | Function         | Status  | Latency (ms) | Return         | Error                                                
----------------------------------------------------------------------------------------------------------------------------------------------------------
5.1.1 | SELECT on price table      | unauthorized_GET | SUCCESS | 157.55       | {'row': '216'} |                                                      
5.1.2 | SELECT on stats table      | unauthorized_GET | SUCCESS | 159.73       | {'row': '220'} |                                                      
5.1.3 | SELECT on price table      | authorized_GET   | SUCCESS | 689.64       | {'row': '216'} |                                                      
5.1.4 | SELECT on stats table      | authorized_GET   | SUCCESS | 703.62       | {'row': '220'} |                                                      
5.1.5 | INSERT on price table      | authorized_POST  | FAILED  | 434.08       | Non

# Latency measurement

In [7]:
measure_latency(unauthorized_GET, 20, "price")
measure_latency(authorized_GET, 20, "stats", credentials)


{'avg_ms': 792.8257942199707,
 'min_ms': 671.7441082000732,
 'max_ms': 1389.228105545044,
 'runs': 20,
 'result': {'row': '220'}}